In [1]:
from base64 import b16decode
from math import ceil
from ipython_genutils.py3compat import buffer_to_bytes
from requests import HTTPError
import streamlit as st
import pandas as pd
import numpy as np
import datetime
import time
from dateutil.parser import parse
from streamlit_option_menu import option_menu
import plotly.express as px
import plotly.graph_objects as go
from torch import true_divide
import requests

2022-07-13 11:51:11.673 INFO    numexpr.utils: NumExpr defaulting to 4 threads.


## BASE PRODUTIVIDADE GERAL ANOS

In [635]:

data = requests.get("http://177.52.21.58:3000/backend/maf/buscarRitmoProducao")
json_data = data.json()
df_piv_2=pd.json_normalize(json_data)
df_piv_2 = pd.DataFrame.from_dict(df_piv_2)

dic = df_piv_2['recordset'][0]

item__ = dic[0] 
df_controle = pd.DataFrame(dic)

df_controle['INICIO_PROCESSAMENTO'] = df_controle['INICIO_PROCESSAMENTO'].str.split('.').str[0]
df_controle['FIM_PROCESSAMENTO'] = df_controle['FIM_PROCESSAMENTO'].str.split('.').str[0]

#df_controle['INICIO_PROCESSAMENTO'] = pd.to_datetime(df_controle['INICIO_PROCESSAMENTO'])
#df_controle['FIM_PROCESSAMENTO'] = pd.to_datetime(df_controle['FIM_PROCESSAMENTO'])
df = df_controle


In [636]:
df = df.dropna()

In [637]:
 def correcao_variedade(df):
    
    if df['VARIEDADE'] == 'TOMMY' or df['VARIEDADE'] == 'TONNY' or df['VARIEDADE'] == 'TOOMY' or df['VARIEDADE'] == 'TOOMY ATKINS' or df['VARIEDADE'] == 'TOMMY,'  or df['VARIEDADE'] == 'TOMMY 2' or df['VARIEDADE'] == 'TOMMT' or df['VARIEDADE'] == 'TOMMY01' or df['VARIEDADE'] == 'tommy' or df['VARIEDADE'] == ' TOMMY' or df['VARIEDADE'] == 'TOMMY ' or df['VARIEDADE'] == 'TOMMY ATKINS' or df['VARIEDADE'] == 'TOMMY  ' or df['VARIEDADE'] == '  TOMMY  '  or df['VARIEDADE'] == '  TOMMY':
        return 'Tommy'
    
    elif df['VARIEDADE'] == 'KEITT' or df['VARIEDADE'] == 'KEIT' or df['VARIEDADE'] == 'KEIIT' or df['VARIEDADE'] == 'KEITT.' or df['VARIEDADE'] == 'KEIITT'  or df['VARIEDADE'] == 'KEITT ' or df['VARIEDADE'] == ' KEITT' or df['VARIEDADE'] == 'KEITT  ' or df['VARIEDADE'] == '  KEITT  ' or df['VARIEDADE'] == '  KEITT' or df['VARIEDADE'] == ' KEITT ':
        return 'Keitt'

    elif df['VARIEDADE'] == 'OALMER' or df['VARIEDADE'] == 'PAMER' or df['VARIEDADE'] == 'PALMER' or df['VARIEDADE'] == 'PALMER' or df['VARIEDADE'] == 'PALMER '  or df['VARIEDADE'] == ' PALMER ' or df['VARIEDADE'] == ' PALER' or df['VARIEDADE'] == ' PALER ' or df['VARIEDADE'] == ' PALER' or df['VARIEDADE'] == 'PALER ': 
        return 'Palmer'

    else:
        return 'remover' 

df['VARIEDADE'] = df.apply(correcao_variedade, axis = 1)

c:\Users\berna\anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [638]:
df

,CONTROLE_MEGA,CONTROLE_MAF,PESO_PROCESSADO,INICIO_PROCESSAMENTO,FIM_PROCESSAMENTO,HORAS_PROCESSAMENTO,TON_HORA,VARIEDADE
0,687,33049,8090.962,2022-07-12T15:43:43,2022-07-12T16:13:09,0.50,16.18,Keitt
1,685,33048,3300.935,2022-07-12T15:10:04,2022-07-12T15:43:44,0.55,6.00,Keitt
2,684,33047,22248.155,2022-07-12T13:57:16,2022-07-12T15:10:06,1.22,18.24,Keitt
3,681,33046,29126.152,2022-07-12T11:04:22,2022-07-12T13:57:18,2.88,10.11,Keitt
4,683,33045,8224.589,2022-07-12T10:29:21,2022-07-12T11:04:24,0.58,14.18,Keitt
...,...,...,...,...,...,...,...,...
1549,2262,31220,6803.169,2021-01-05T14:24:28,2021-01-05T14:48:57,0.40,17.01,Keitt
1550,2259,31219,20470.672,2021-01-05T12:07:34,2021-01-05T14:24:33,2.28,8.98,Keitt
1551,2256,31218,29054.045,2021-01-05T10:30:51,2021-01-05T12:07:39,1.62,17.93,Keitt
1552,2,31217,12655.951,2021-01-05T09:13:48,2021-01-05T10:30:53,1.28,9.89,Tommy


In [639]:
filtro_osteen = df['VARIEDADE'] != 'remover'
df = df[filtro_osteen]

In [640]:
df['DATA'] = df['INICIO_PROCESSAMENTO'].str.split('T').str[0]
df['ANO'] = df['DATA'].str.split('-').str[0]
df['MES'] = df['DATA'].str.split('-').str[1]
df['DIA'] = df['DATA'].str.split('-').str[2]

c:\Users\berna\anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [642]:
df['MES'] = df['MES'].astype(int)
df['DIA'] = df['DIA'].astype(int)

c:\Users\berna\anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [643]:
def remocao_mes(df):
    if df['MES'] == 4 or df['MES'] == 5 or df['MES'] == 6 or df['MES'] == 7:
        return 'manter'
    else:
        return 'remover'

df['out'] = df.apply(remocao_mes, axis = 1)

In [644]:
df_filtro  = df['out'] == 'manter'
df = df[df_filtro]
        

In [645]:
df_piv_dia_mes = pd.pivot_table(df, index = ['ANO','MES','DIA','VARIEDADE'], values = 'TON_HORA',aggfunc = np.mean)
df_piv_dia_mes = df_piv_dia_mes.reset_index()

In [648]:
df['MES'] = df['MES'].astype(int)
df['DIA'] = df['DIA'].astype(int)

fig = px.box(df,x = 'MES',y = 'TON_HORA', color = 'ANO', hover_name = 'DIA', facet_row = 'VARIEDADE',hover_data=["CONTROLE_MEGA"],points="all")
fig.update_layout(height = 1200, width = 1800)
fig

c:\Users\berna\anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [657]:
fig = px.histogram(df,x = 'MES',y = 'TON_HORA', histfunc  = 'avg',color = 'ANO', facet_col = 'VARIEDADE' )
fig.update_layout(height = 500, width = 1000)
fig

In [655]:
fig = px.line(df_piv_dia_mes,x = 'DIA',y = 'TON_HORA', color = 'ANO', facet_col =  'MES', facet_row = 'VARIEDADE' )
fig.update_layout(height = 600, width = 1250, hovermode="x unified")
fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.update_xaxes(categoryorder='category ascending')
fig

## BASE DE PRODUTIVIDADE RECENTE EMBALADEIRAS INDIVIDUAIS

In [658]:
data2 = requests.get("http://177.52.21.58:3000/backend/busca_generica/buscaGenericaMYSQLAvilla?sql=CALL%209090agrodan.SP_DX_EMB_PRODUTIVIDADE_RECENTE()")
json_data2 = data2.json()
df_piv_3=pd.json_normalize(json_data2)
df_piv_3 = pd.DataFrame.from_dict(df_piv_3)

dic = df_piv_3[:1]

dic = dic.reset_in dex()

df_22 = dic.set_index('index').T.to_dict('list')

aa = df_22.get(0)

df_ = pd.DataFrame.from_dict(aa)
df_


,LEH_DATA,VARIEDADE,CALIBRE,LEH_QUANTIDADE,LINHAPRODUCAO,HORAS,CX_HORA,ID_PESSOA,PESSOA,MATRICULA
0,2022-07-12T10:06:18.000Z,Palmer -02,10,17,16,0.1500,113.333333,956,002 - Luciene Ildete S. Lopes,00851365493
1,2022-07-12T10:16:00.000Z,Palmer -02,10,21,17,0.2333,90.000000,956,002 - Luciene Ildete S. Lopes,00851365493
2,2022-07-12T10:30:34.000Z,Palmer -02,10,23,17,0.2333,98.571429,956,002 - Luciene Ildete S. Lopes,00851365493
3,2022-07-12T10:45:33.000Z,Palmer -02,10,16,17,0.2333,68.571429,956,002 - Luciene Ildete S. Lopes,00851365493
4,2022-07-12T11:00:04.000Z,Palmer -02,10,17,17,0.2500,68.000000,956,002 - Luciene Ildete S. Lopes,00851365493
...,...,...,...,...,...,...,...,...,...,...
1166,2022-07-12T18:00:15.000Z,Tommy - 30,8,5,16,0.0667,75.000001,1566,249 - Irailde Maria Martins Santos,06156229418
1167,2022-07-12T18:05:14.000Z,Tommy - 29,7,2,22,0.3000,6.666667,1566,249 - Irailde Maria Martins Santos,06156229418
1168,2022-07-12T18:23:52.000Z,Tommy - 29,7,15,22,0.7333,20.454545,1566,249 - Irailde Maria Martins Santos,06156229418
1169,2022-07-12T19:08:49.000Z,Tommy - 30,8,3,16,0.1167,25.714286,1566,249 - Irailde Maria Martins Santos,06156229418


In [659]:
df_['LEH_DATA'] = df_['LEH_DATA'].str.split('.').str[0]

In [663]:
df_['DATA'] = df_['LEH_DATA'].str.split('T').str[0]
df_['ANO'] = df_['DATA'].str.split('-').str[0]
df_['MES'] = df_['DATA'].str.split('-').str[1]
df_['DIA'] = df_['DATA'].str.split('-').str[2]

In [664]:
df_

,LEH_DATA,VARIEDADE,CALIBRE,LEH_QUANTIDADE,LINHAPRODUCAO,HORAS,CX_HORA,ID_PESSOA,PESSOA,MATRICULA,DATA,ANO,MES,DIA
0,2022-07-12T10:06:18,Palmer -02,10,17,16,0.1500,113.333333,956,002 - Luciene Ildete S. Lopes,00851365493,2022-07-12,2022,07,12
1,2022-07-12T10:16:00,Palmer -02,10,21,17,0.2333,90.000000,956,002 - Luciene Ildete S. Lopes,00851365493,2022-07-12,2022,07,12
2,2022-07-12T10:30:34,Palmer -02,10,23,17,0.2333,98.571429,956,002 - Luciene Ildete S. Lopes,00851365493,2022-07-12,2022,07,12
3,2022-07-12T10:45:33,Palmer -02,10,16,17,0.2333,68.571429,956,002 - Luciene Ildete S. Lopes,00851365493,2022-07-12,2022,07,12
4,2022-07-12T11:00:04,Palmer -02,10,17,17,0.2500,68.000000,956,002 - Luciene Ildete S. Lopes,00851365493,2022-07-12,2022,07,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,2022-07-12T18:00:15,Tommy - 30,8,5,16,0.0667,75.000001,1566,249 - Irailde Maria Martins Santos,06156229418,2022-07-12,2022,07,12
1167,2022-07-12T18:05:14,Tommy - 29,7,2,22,0.3000,6.666667,1566,249 - Irailde Maria Martins Santos,06156229418,2022-07-12,2022,07,12
1168,2022-07-12T18:23:52,Tommy - 29,7,15,22,0.7333,20.454545,1566,249 - Irailde Maria Martins Santos,06156229418,2022-07-12,2022,07,12
1169,2022-07-12T19:08:49,Tommy - 30,8,3,16,0.1167,25.714286,1566,249 - Irailde Maria Martins Santos,06156229418,2022-07-12,2022,07,12


WE DID !!!! 

In [596]:
df_.VARIEDADE.value_counts()

Palmer -02    860
Keitt 02 C     64
Tommy - 31     59
Tommy - 32     47
Tommy - 30     41
Tommy - 29     23
Linha 14        5
Name: VARIEDADE, dtype: int64

In [597]:
def correcao_nomes_variedades(df_):
    if df_['VARIEDADE'] == 'Tommy - 31' or df_['VARIEDADE'] == 'Tommy - 30' or df_['VARIEDADE'] == 'Tommy - 32' or df_['VARIEDADE'] == 'Tommy - 29':
        return 'Tommy Atkins'
    elif df_['VARIEDADE'] == 'Palmer -02':
        return 'Palmer'
    elif df_['VARIEDADE'] == 'Keitt 02 C' or df_['VARIEDADE'] == 'Keitt 02':
        return 'Keitt'
    else:
        return 'NADA'
df_['VARIEDADE'] = df_.apply(correcao_nomes_variedades, axis = 1)
    

In [598]:
df_.VARIEDADE.value_counts()

Palmer          860
Tommy Atkins    170
Keitt            64
NADA              5
Name: VARIEDADE, dtype: int64

In [599]:
df_['LEH_DATA'] = df_['LEH_DATA'].str.split('.').str[0]

In [600]:
df_['LEH_DATA'] = pd.to_datetime(df_['LEH_DATA'])